In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.ndimage import shift
mnist = fetch_openml('mnist_784', as_frame=False)

/usr/local/lib/python3.9/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
X, y = mnist.data, mnist.target

In [ ]:
X.shape

(70000, 784)

In [ ]:
y

array(['5', '0', '4', ..., '4', '5', '6'], dtype=object)

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
param_grid = {'n_neighbors': [3, 5, 7, 9],
              'weights': ['uniform', 'distance']}

In [ ]:
knn_set = KNeighborsClassifier()
knn_set.fit(X_train, y_train)
baseline_accuracy = knn_set.score(X_test, y_test)
baseline_accuracy

0.9688

In [ ]:
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train[:10000], y_train[:10000])

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 7, 9],
                         'weights': ['uniform', 'distance']})

In [ ]:
grid_search.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

In [ ]:
grid_search.best_score_

0.9423999999999999

In [ ]:
knn = grid_search.best_estimator_
knn.fit(X_train,y_train) 
tuned_accuracy = grid_search.score(X_test, y_test)
tuned_accuracy

0.9717

In [ ]:
import numpy as np
from scipy.ndimage import shift

def mass_shift_mnist_images(images, direction):
    
    images = images.reshape((28,28))
    if direction == "left":
        shift_amount = (-1, 0)
    if direction == "right":
        shift_amount = (1, 0)
    if direction == "up":
        shift_amount = (0, -1)
    if direction == "down":
        shift_amount = (0, 1)

    shifted_image = shift(images, shift_amount, cval=0, mode="constant")
    
    return shifted_image.reshape([-1])

In [ ]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]
for direction in ("left","right","up","down"):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(mass_shift_mnist_images(image,direction))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [ ]:
shuffle = np.random.permutation(len(X))
X_train_augmented = X_train_augmented[shuffle]
y_train_augmented = y_train_augmented[shuffle]

In [ ]:
knn.fit(X_train_augmented,y_train_augmented)
augmented_accuracy = knn.score(X_test,y_test)
augmented_accuracy

0.9726

In [ ]:
error_rate_change = (1 - augmented_accuracy) / (1 - tuned_accuracy) - 1
print(f"error_rate_change = {error_rate_change:.0%}")

error_rate_change = -3%
